Подгружаем необходимые библиотеки и данные

In [ ]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords   
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

train = pd.read_csv('../input/bag-of-words-meets-bags-of-popcorn-fixed-zips/labeledTrainData.tsv',
                    header=0,
                    delimiter="\t",
                    quoting=3)
test = pd.read_csv('../input/bag-of-words-meets-bags-of-popcorn-fixed-zips/testData.tsv',
                   header=0,
                   delimiter="\t",
                   quoting=3 )

Напишем функцию которая преобразует текст превью строку из слов. Уберём html-код и символы не являющимися словами. Результат работы функции: строка из слов разделённых пробелами.

In [ ]:
def review_to_words( raw_review ):

    review_text = BeautifulSoup(raw_review).get_text() 
 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 

    words = letters_only.lower().split()                             

    stops = set(stopwords.words("english"))                  

    meaningful_words = [w for w in words if not w in stops]   

    return( " ".join( meaningful_words )) 

Вызовем функцию для каждого ревью

In [ ]:
num_reviews = train["review"].size

clean_train_reviews = []

for i in range( 0, num_reviews ):
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

Используем CountVectorizer. Обучаем и преобразуем тренировочные данные в лист векторов. После этого можно преобразовать всё в удобный для работы формат 0 - numpy массив.

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

train_data_features.shape

Выведем полученный словарь

In [ ]:
vocab = vectorizer.get_feature_names()
len(vocab)

В качестве классификатора используем случайный лес. Заполним лист "очищенными" ревью. Преобразуем в numpy массив, после этого можно предсказывать.

In [ ]:
forest = RandomForestClassifier(n_jobs=-1, n_estimators=500)

forest = forest.fit( train_data_features, train["sentiment"] )

num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "./submission.csv", index=False, quoting=3 )